In [11]:
import pygame
import numpy as np

In [12]:
class Node:
    def __init__(self, _pos, _walkable=True, _explored=False,_visited=False, _parent_node = None):
        self.pos = _pos
        self.walkable = _walkable
        self.cost = 1e6
        self.parent_node = _parent_node
        self.explored = _explored
        self.visited = _visited

In [13]:
class Field:
    def __init__(self, _dimensions=[20,20]):
        self.dimensions = _dimensions
        self.board = [[ Node([i,j]) for j in range(self.dimensions[1])] for i in range(self.dimensions[0])]
        self.to_explore = []
        self.path = []
        
    
    def not_walkable_area(self,x,y):
        # Set the element of the grid to not walkable
        self.board[x][y].walkable = False
    
    def neightbours_cost(self, gui, initial_node, start, end):
        # Determine the costs f the sourrounding 
        for i in range(-1,2,1):
            for j in range(-1,2,1):
                
                # Omit the current node
                if i == 0 and j == 0:
                    continue
                # Check that the next position lies within the boundaries
                if not 0 <= i+initial_node.pos[0] < self.dimensions[0] or not 0 <= j+initial_node.pos[1] < self.dimensions[1]:
                    continue
                # Check that the next position is accesible
                if not self.board[i+initial_node.pos[0]][j+initial_node.pos[1]].walkable:
                    continue
                    
                # Compute the cost (distance) of the sourroundings nodes
                cost_start_vector = start - (np.asarray([i,j])+np.asarray(initial_node.pos))
                cost_end_vector = end - (np.asarray([i,j])+np.asarray(initial_node.pos))
                cost_start = np.sqrt(np.sum(cost_start_vector**2))
                cost_end = np.sqrt(np.sum(cost_end_vector**2))

                # Set the parent of the explored node and ad to the explored list  
                if not self.board[i+initial_node.pos[0]][j+initial_node.pos[1]].explored: 
                    self.board[i+initial_node.pos[0]][j+initial_node.pos[1]].explored = True
                    self.board[i+initial_node.pos[0]][j+initial_node.pos[1]].parent_node = self.board[initial_node.pos[0]][initial_node.pos[1]] 
                    self.board[i+initial_node.pos[0]][j+initial_node.pos[1]].cost = cost_start + 5*cost_end
                    self.to_explore.append(self.board[i+initial_node.pos[0]][j+initial_node.pos[1]])
                    
                    # Draw movement
                    h, w = i+initial_node.pos[0], j+initial_node.pos[1]
                    pygame.draw.rect(gui.screen,(255,255,0),(h*gui.unit_size+1,w*gui.unit_size+1,gui.unit_size-2,gui.unit_size-2))
        pygame.display.update()

        
    def create_path(self,gui,initial_pos,end_pos):
        #list of positions that the path is following
        path_found = False
        # Find the node at the initial position and expllore it
        current_node = self.board[initial_pos[0]][initial_pos[1]]
        current_node.explored = True
        current_node.visited = True
        # the initial node have been added to the to explore list for consistency in the next loop
        self.to_explore.append(current_node)
        
        # path loop
        while not path_found:
            pygame.time.wait(0)
            # Determine the cost of the neightbor elements            
            self.neightbours_cost(gui, current_node,initial_pos,end_pos)
            
            # Remove element from explored list
            self.to_explore.remove(current_node)
            
            # Stablish cost for neightbours that have not been visited yet
            cost = 1e6
            for node in self.to_explore:
                if node.cost < cost and not node.visited:
                    current_node = node
                    cost = current_node.cost
            current_node.visited = True
            
            # Draw movement
            [h, w] = current_node.pos
            pygame.draw.rect(gui.screen,(255,120,0),(h*gui.unit_size+1,w*gui.unit_size+1,gui.unit_size-2,gui.unit_size-2))
            pygame.display.update()
            
            # Check if the end of the path have been reached
            if current_node.pos == end_pos:
                path_found = True
                
        # write actual path to the path position list 
        start_found = False
        while not start_found:
            self.path.insert(0,current_node)
            current_node = current_node.parent_node
            if current_node.pos == initial_pos:
                self.path.insert(0,current_node)
                start_found = True
            

In [38]:
class GUI:
    def __init__(self, _game, _field_size, _unit_size=20):
        self.height, self.width, self.unit_size = _field_size[0], _field_size[1], _unit_size
        self.cfill = (0,0,0) # Black color
        # Start GraphicalUserInterface
        pygame.init()
        self.screen = pygame.display.set_mode((self.height*self.unit_size, self.width*self.unit_size))
        self.screen.fill((0,0,0))
        pygame.display.set_caption('aStar Visualization')
        
    def forbiden_positions(self,game,start,end):
        
        for h in range(self.height):
            for  w in range(self.width):
                if (h ==start[0] and w == start[1]) or (h == end[0] and w==end[1]):
                    pygame.draw.rect(self.screen,(250,0,0),(h*self.unit_size+1,w*self.unit_size+1,self.unit_size-2,self.unit_size-2))
                else:
                    pygame.draw.rect(self.screen,(250,250,250),(h*self.unit_size+1,w*self.unit_size+1,self.unit_size-2,self.unit_size-2))
        pygame.display.update()
        
        # Wait till the user has draw all the walls
        finish = False
        while not finish:
            for event in pygame.event.get():
                # Draw walls given by the user
                if pygame.mouse.get_pressed()[0]:
                    event_pos = pygame.mouse.get_pos()
                    x = int(event_pos[0]/self.unit_size)
                    y = int(event_pos[1]/self.unit_size)
                    game.board[x][y].walkable = False
                    pygame.draw.rect(self.screen,(0,0,0),(x*self.unit_size+1,y*self.unit_size+1,self.unit_size-2,self.unit_size-2))
                    pygame.display.update()
                # Possible keys to start searching for the best path given the initial and final positons
                if event.type == pygame.KEYUP and event.key == pygame.K_ESCAPE:
                    finish = True
                    break
                if event.type == pygame.KEYUP and event.key == pygame.K_RETURN:
                    finish = True
                    break
                if event.type == pygame.KEYUP and event.key == pygame.K_SPACE:
                    finish = True
                    break

    # Draw grid with at the end of the game
    def draw_grid(self, game, start, end):
        for h in range(self.height):
            for w in range(self.width):
                if game.board[h][w] in game.path:
                    pygame.draw.rect(self.screen,(255,0,0),(h*self.unit_size+1,w*self.unit_size+1,self.unit_size-2,self.unit_size-2))
                elif not game.board[h][w].walkable:
                    pygame.draw.rect(self.screen,(0,0,0),(h*self.unit_size+1,w*self.unit_size+1,self.unit_size-2,self.unit_size-2))
                elif game.board[h][w].visited:
                    pygame.draw.rect(self.screen,(255,120,60),(h*self.unit_size+1,w*self.unit_size+1,self.unit_size-2,self.unit_size-2))
                elif game.board[h][w].explored:
                    pygame.draw.rect(self.screen,(255,255,0),(h*self.unit_size+1,w*self.unit_size+1,self.unit_size-2,self.unit_size-2))
                else:
                    pygame.draw.rect(self.screen,(250,250,250),(h*self.unit_size+1,w*self.unit_size+1,self.unit_size-2,self.unit_size-2))
        pygame.draw.rect(self.screen,(250,10,0),(start[0]*self.unit_size+1,start[1]*self.unit_size+1,self.unit_size-2,self.unit_size-2))
        pygame.draw.rect(self.screen,(250,10,0),(end[0]*self.unit_size+1,end[1]*self.unit_size+1,self.unit_size-2,self.unit_size-2))
        pygame.display.update()
        
    def close_GUI(self):
        pygame.display.quit()

In [39]:
field = [40,20]
game = Field(field)

start = [np.random.randint(0,field[0]),np.random.randint(0,field[1])]
end = [np.random.randint(0,field[0]),np.random.randint(0,field[1])]
g = GUI(game,field)

g.forbiden_positions(game,start,end)
game.create_path(g,start,end)

g.draw_grid(game,start,end)
pygame.time.wait(3000)
pygame.display.quit()

In [32]:
pygame.display.quit()
